In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import src.data_loader as dl
import src.trade as trade
import src.metrics as metrics
import src.cross_validation as cv
import time
import numpy as np
from importlib import reload 
import src.telegram as telegram
import os
from dotenv import load_dotenv
import sys
import pandas as pd
import src.back_test as bt

load_dotenv()

True

In [46]:
reload(cv)
reload(bt)

<module 'src.back_test' from '/Users/thiagocunha/git/LS/src/back_test.py'>

In [43]:
SHEET_PATH = os.getenv('SHEET_PATH')

In [4]:
#ranges de valores
portfolio_max_size = np.arange(2,4,1)
dickey_fuller = np.arange(3,4,1)
fisher = np.arange(-0.5,-0.4,0.1)
meia_vida = [10,25]
media_n = np.arange(0.5, 0.6, 0.1)
desvio_padrao = np.arange(2, 2.1, 0.1)
periodo = [100]
variancia_beta = np.arange(0.01, 0.12, 0.1)
dagostino_person = 0.05

In [47]:
df = dl.sheet_importer(SHEET_PATH)

Intervalo do dataset: 2015-01-02 00:00:00 a 2020-07-14 00:00:00
Shape: (9324, 27)


In [6]:
grid = cv.generate_grid(portfolio_max_size, dickey_fuller, fisher, meia_vida,  media_n,  desvio_padrao, periodo, variancia_beta)

In [48]:
cv.grid_search_info(df, grid, parallel=True, send_to_telegram=True)

Total de iterações: 16
Uso de memória: 2.304e-06 Gb
-------------------------
Quantidade de iterações em um ciclo: 8
Tempo de um ciclo: 58.18 segundos
-------------------------
Tempo previsto da busca: 0.03 horas
Tempo previsto da busca: 0.0 dias


In [36]:
try:
    results = cv.grid_search(df, grid, parallel=True, verbose=True, send_to_telegram=False)
except:
    telegram.send("{} ERROR: {}".format(telegram.emoji.double_exclamation,sys.exc_info()))

--- Inicio do Grid Search ---
--- Fim do Grid Search ---


In [50]:
df_results = pd.DataFrame(results)
df_results.head()

,iteracao,portfolio_max_size,dickey-fuller,fisher,meia-vida,media_n,desvio-padrao,periodo,variancia_beta,dagostino-person,n_operacoes,retorno,win_rate,pay_off,mdd,recovery_factor
0,2,2,3,-0.5,10,0.5,2.1,100,0.01,0.05,441,33526.0,0.59,0.820,0.934,1.178
1,0,2,3,-0.5,10,0.5,2.0,100,0.01,0.05,469,39618.0,0.60,0.810,1.027,1.200
2,3,2,3,-0.5,10,0.5,2.1,100,0.11,0.05,450,39626.0,0.59,0.834,0.843,1.206
3,1,2,3,-0.5,10,0.5,2.0,100,0.11,0.05,474,35946.0,0.59,0.799,1.071,1.174
4,6,2,3,-0.5,25,0.5,2.1,100,0.01,0.05,625,15338.0,0.58,0.774,0.803,1.052


In [52]:
df.to_pickle("./result.pkl")

In [53]:
df2 = pd.read_pickle("./result.pkl")


In [54]:
df2

,data,pares,pares_invertidos,ativo_dep,ativo_ind,dickey-fuller,adf,prop_financeiro,fisher,meia-vida,...,media_n,data_saida,in_dep,in,in.1,out,out.1,res,res.1,resultado
0,2015-01-02,ITSA4|BBDC3,BBDC3|ITSA4,5700,-1800,3,-4.289345,1.17,0.824511,19,...,0.052786,2015-01-09,6,4.77,13.27,4.86,13.61,513.0,-612.0,-99.0
1,2015-01-02,IGTA3|BRML3,BRML3|IGTA3,1400,-1900,9,-6.626779,1.59,0.664409,12,...,0.281100,2015-01-05,2,21.20,9.63,21.71,9.22,714.0,779.0,1493.0
2,2015-01-02,RENT3|CCRO3,CCRO3|RENT3,-2900,1700,4,-4.252567,1.42,0.480409,19,...,-0.028646,2015-01-07,4,9.99,12.15,9.63,12.16,1044.0,17.0,1061.0
3,2015-01-02,JBSS3|TRPL4,TRPL4|JBSS3,2900,-3100,3,-4.028523,1.34,0.155227,16,...,0.149986,2015-01-09,6,9.90,6.98,9.56,6.62,-986.0,1116.0,130.0
4,2015-01-02,JBSS3|LAME4,LAME4|JBSS3,3300,-1300,4,-4.370029,1.85,0.347030,19,...,0.066820,2015-01-09,6,9.90,13.67,9.56,13.33,-1122.0,442.0,-680.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9319,2020-07-14,MRVE3|CSNA3,CSNA3|MRVE3,-1400,1700,5,-4.708377,1.47,0.385239,7,...,-0.066807,2020-07-21,6,20.81,11.65,20.61,11.63,280.0,-34.0,246.0
9320,2020-07-14,LREN3|CYRE3,CYRE3|LREN3,700,-700,8,-5.140300,1.48,0.675353,13,...,-1.133676,2020-07-21,6,42.00,26.65,42.73,27.13,511.0,-336.0,175.0
9321,2020-07-14,COGN3|EQTL3,EQTL3|COGN3,-2000,1400,6,-5.010727,0.54,0.418118,10,...,-0.121981,2020-07-21,6,8.71,23.85,9.42,23.89,-1420.0,56.0,-1364.0
9322,2020-07-14,COGN3|VVAR3,VVAR3|COGN3,-2600,1500,4,-4.164408,0.85,0.536048,16,...,-0.955506,2020-07-21,6,8.71,17.99,9.42,20.81,-1846.0,4230.0,2384.0
